In [18]:
###用于测试csv和mlp
import torch
import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

import time
from collections import namedtuple

from torch.utils.data.sampler import WeightedRandomSampler

import torch.nn as nn

from sklearn.metrics import confusion_matrix

# 五层神经网络
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

mlp = NeuralNet(14,200,4)
print(mlp)

##############################################读写CSV
file1 = "./trainData/dataAllSim.csv"
##方式1
data1=[]
csv_reader = csv.reader(open(file1))
#for row in csv_reader:
#        data1.append(row)
#print(data1[0])        

##方式2
#https://blog.csdn.net/zw__chen/article/details/82806900
csvfile = open(file1, 'r')
xyData= np.loadtxt(file1,delimiter=',',skiprows=1)
x = xyData[:,0:-1]
y = xyData[:,-1]
print(y.shape )
print(x.shape )
x1 =torch.from_numpy(x)
y1 =torch.from_numpy(y)

dataset1 = TensorDataset(x1.float(),y1.long())

#train_dataset, test_dataset = random_split(
#    dataset=dataset1,
#    lengths=[7, 3],
#    generator=torch.Generator().manual_seed(0)
#)

#################################
#数据准备：平衡
#tmp = y.T[0]
#trainratio = np.bincount(tmp.astype("int64"))
#train_weights = 1./torch.tensor(trainratio, dtype=torch.float)
#train_sampleweights = train_weights[y]
#train_sampler = WeightedRandomSampler(weights=train_sampleweights, num_samples = len(train_sampleweights))
#train_loader = DataLoader( dataset1, batch_size=4, num_workers=1, sampler=train_sampler)
train_loader = DataLoader(dataset=dataset1, batch_size=512, shuffle=True)

#准备设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cuda':
    mlp = torch.nn.DataParallel(mlp)
    cudnn.benchmark = True
    mlp =  mlp.to(device)
print(device)

#准备模型
state = {"net": mlp.state_dict(), "optimizer": optimizer.state_dict()}
modelPathName = "./trainedModes/"+"redTLS.modeparams"
params = torch.load(modelPathName)
mlp.load_state_dict(params['net'])
optimizer.load_state_dict(params['optimizer'])

#################################
#训练开始
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
running_loss = 0
startEpoch = 0
epochs = 3000#大概8小时
print("start training")

for epoch in range(startEpoch, epochs):  # 多批次循环

     mlp.train()
     time_start = time.time()
     total = 0
     correct = 0
     running_loss = 0.0
     
     for i, data in enumerate(train_loader, 0):
         # 获取输入
         #print(epoch,i)
         inputs, labels = data
         #inputs, labels = inputs.to(device), labels.to(device)
         inputs=inputs.type(torch.float32)
         labels=labels
           
         if device == 'cuda':
            inputs, labels = inputs.to(device), labels.to(device)

         # 梯度置0
         optimizer.zero_grad()

         # 正向传播，反向传播，优化a
         outputs = mlp(inputs)
         #print("inputs:",inputs)
         #print("###################################")
         #print("outputs:",outputs)
         #print("labels",labels)

         loss = criterion(outputs, labels)
         #print("loss:",loss.item())
        
         loss.backward()
         optimizer.step()

         _, predicted = torch.max(outputs.data,1)
         total += labels.size(0)
         correct += predicted.eq(labels).sum().item()
         
         #print("labels.size(0):",labels.size(0))
         #print("predicted:",predicted)
         #print("predicted.eq(labels).sum().item():",predicted.eq(labels).sum().item())
         
         running_loss += loss.item()
         
         #os.input()
         if i % 200 == 1:  # 每200批次打印一次
             time_end = time.time()
             print('##########one 200 batch totally time cost %.3f' %(time_end-time_start))
             print("batchIndex %d |trainLen %d | Loss: %.3f | Acc: %.3f | correct, total: (%d,%d)" % (
                 i, len(train_loader), running_loss/(i+1), 100.*correct/total, correct, total))
             
     time_end = time.time()
     print("epochIndex %d |time: %d | Acc: %.3f | correct, total: (%d,%d)" % (
                 epoch, (time_end-time_start), 100.*correct/total, correct, total))
     print("########################################################################")
     
    
state = {"net": mlp.state_dict(), "optimizer": optimizer.state_dict()}
modelPathName = "./trainedModes/"+"redTLS.modeparams"
torch.save(state, modelPathName)
params = torch.load(modelPathName)
mlp.load_state_dict(params['net'])
optimizer.load_state_dict(params['optimizer'])

NeuralNet(
  (fc1): Linear(in_features=14, out_features=200, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=4, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
(723532,)
(723532, 14)
start training
##########one 200 batch totally time cost 0.058
batchIndex 1 |trainLen 1414 | Loss: 1.187 | Acc: 46.191 | correct, total: (473,1024)
##########one 200 batch totally time cost 1.497
batchIndex 201 |trainLen 1414 | Loss: 0.518 | Acc: 82.058 | correct, total: (84868,103424)
##########one 200 batch totally time cost 2.980
batchIndex 401 |trainLen 1414 | Loss: 0.441 | Acc: 83.959 | correct, total: (172807,205824)
##########one 200 batch totally time cost 4.891
batchIndex 601 |trainLen 1414 | Loss: 0.406 | Acc: 84.916 | correct, total: (261733,308224)
##########one 200 batch totally time cost 6.291
batchIndex 801 |trainLen 1414 | Loss: 0.380 

##########one 200 batch totally time cost 4.328
batchIndex 601 |trainLen 1414 | Loss: 0.201 | Acc: 91.713 | correct, total: (282680,308224)
##########one 200 batch totally time cost 5.739
batchIndex 801 |trainLen 1414 | Loss: 0.200 | Acc: 91.740 | correct, total: (376708,410624)
##########one 200 batch totally time cost 7.104
batchIndex 1001 |trainLen 1414 | Loss: 0.199 | Acc: 91.791 | correct, total: (470910,513024)
##########one 200 batch totally time cost 8.534
batchIndex 1201 |trainLen 1414 | Loss: 0.197 | Acc: 91.854 | correct, total: (565292,615424)
##########one 200 batch totally time cost 9.938
batchIndex 1401 |trainLen 1414 | Loss: 0.198 | Acc: 91.831 | correct, total: (659183,717824)
epochIndex 6 |time: 10 | Acc: 91.833 | correct, total: (664444,723532)
########################################################################
##########one 200 batch totally time cost 0.062
batchIndex 1 |trainLen 1414 | Loss: 0.183 | Acc: 92.676 | correct, total: (949,1024)
##########one 200 ba

##########one 200 batch totally time cost 1.536
batchIndex 201 |trainLen 1414 | Loss: 0.184 | Acc: 92.213 | correct, total: (95370,103424)
##########one 200 batch totally time cost 2.881
batchIndex 401 |trainLen 1414 | Loss: 0.190 | Acc: 92.075 | correct, total: (189512,205824)
##########one 200 batch totally time cost 4.303
batchIndex 601 |trainLen 1414 | Loss: 0.187 | Acc: 92.178 | correct, total: (284115,308224)
##########one 200 batch totally time cost 5.710
batchIndex 801 |trainLen 1414 | Loss: 0.186 | Acc: 92.230 | correct, total: (378720,410624)
##########one 200 batch totally time cost 7.062
batchIndex 1001 |trainLen 1414 | Loss: 0.187 | Acc: 92.185 | correct, total: (472932,513024)
##########one 200 batch totally time cost 8.521
batchIndex 1201 |trainLen 1414 | Loss: 0.189 | Acc: 92.122 | correct, total: (566941,615424)
##########one 200 batch totally time cost 9.908
batchIndex 1401 |trainLen 1414 | Loss: 0.188 | Acc: 92.110 | correct, total: (661185,717824)
epochIndex 13 |tim

##########one 200 batch totally time cost 8.529
batchIndex 1201 |trainLen 1414 | Loss: 0.183 | Acc: 92.332 | correct, total: (568235,615424)
##########one 200 batch totally time cost 9.901
batchIndex 1401 |trainLen 1414 | Loss: 0.183 | Acc: 92.338 | correct, total: (662827,717824)
epochIndex 19 |time: 10 | Acc: 92.343 | correct, total: (668128,723532)
########################################################################
##########one 200 batch totally time cost 0.066
batchIndex 1 |trainLen 1414 | Loss: 0.238 | Acc: 90.430 | correct, total: (926,1024)
##########one 200 batch totally time cost 1.502
batchIndex 201 |trainLen 1414 | Loss: 0.187 | Acc: 92.227 | correct, total: (95385,103424)
##########one 200 batch totally time cost 2.875
batchIndex 401 |trainLen 1414 | Loss: 0.185 | Acc: 92.229 | correct, total: (189829,205824)
##########one 200 batch totally time cost 4.282
batchIndex 601 |trainLen 1414 | Loss: 0.184 | Acc: 92.287 | correct, total: (284450,308224)
##########one 200 bat

##########one 200 batch totally time cost 2.902
batchIndex 401 |trainLen 1414 | Loss: 0.183 | Acc: 92.262 | correct, total: (189898,205824)
##########one 200 batch totally time cost 4.320
batchIndex 601 |trainLen 1414 | Loss: 0.181 | Acc: 92.362 | correct, total: (284683,308224)
##########one 200 batch totally time cost 5.749
batchIndex 801 |trainLen 1414 | Loss: 0.180 | Acc: 92.388 | correct, total: (379366,410624)
##########one 200 batch totally time cost 7.169
batchIndex 1001 |trainLen 1414 | Loss: 0.179 | Acc: 92.423 | correct, total: (474151,513024)
##########one 200 batch totally time cost 8.592
batchIndex 1201 |trainLen 1414 | Loss: 0.179 | Acc: 92.440 | correct, total: (568895,615424)
##########one 200 batch totally time cost 10.058
batchIndex 1401 |trainLen 1414 | Loss: 0.179 | Acc: 92.425 | correct, total: (663447,717824)
epochIndex 26 |time: 10 | Acc: 92.425 | correct, total: (668726,723532)
########################################################################
##########o

...

##########one 200 batch totally time cost 4.294
batchIndex 601 |trainLen 1414 | Loss: 0.175 | Acc: 92.587 | correct, total: (285375,308224)
##########one 200 batch totally time cost 5.716
batchIndex 801 |trainLen 1414 | Loss: 0.176 | Acc: 92.578 | correct, total: (380149,410624)
##########one 200 batch totally time cost 7.111
batchIndex 1001 |trainLen 1414 | Loss: 0.175 | Acc: 92.597 | correct, total: (475047,513024)
##########one 200 batch totally time cost 8.516
batchIndex 1201 |trainLen 1414 | Loss: 0.175 | Acc: 92.609 | correct, total: (569938,615424)
##########one 200 batch totally time cost 9.926
batchIndex 1401 |trainLen 1414 | Loss: 0.175 | Acc: 92.608 | correct, total: (664760,717824)
epochIndex 72 |time: 10 | Acc: 92.608 | correct, total: (670046,723532)
########################################################################
##########one 200 batch totally time cost 0.059
batchIndex 1 |trainLen 1414 | Loss: 0.192 | Acc: 92.090 | correct, total: (943,1024)
##########one 200 b

##########one 200 batch totally time cost 1.470
batchIndex 201 |trainLen 1414 | Loss: 0.175 | Acc: 92.551 | correct, total: (95720,103424)
##########one 200 batch totally time cost 2.882
batchIndex 401 |trainLen 1414 | Loss: 0.175 | Acc: 92.648 | correct, total: (190691,205824)
##########one 200 batch totally time cost 4.293
batchIndex 601 |trainLen 1414 | Loss: 0.174 | Acc: 92.680 | correct, total: (285661,308224)
##########one 200 batch totally time cost 5.723
batchIndex 801 |trainLen 1414 | Loss: 0.176 | Acc: 92.616 | correct, total: (380302,410624)
##########one 200 batch totally time cost 7.153
batchIndex 1001 |trainLen 1414 | Loss: 0.176 | Acc: 92.595 | correct, total: (475036,513024)
##########one 200 batch totally time cost 8.510
batchIndex 1201 |trainLen 1414 | Loss: 0.176 | Acc: 92.580 | correct, total: (569761,615424)
##########one 200 batch totally time cost 9.939
batchIndex 1401 |trainLen 1414 | Loss: 0.176 | Acc: 92.587 | correct, total: (664610,717824)
epochIndex 79 |tim

##########one 200 batch totally time cost 8.946
batchIndex 1201 |trainLen 1414 | Loss: 0.175 | Acc: 92.596 | correct, total: (569855,615424)
##########one 200 batch totally time cost 10.812
batchIndex 1401 |trainLen 1414 | Loss: 0.174 | Acc: 92.622 | correct, total: (664866,717824)
epochIndex 85 |time: 10 | Acc: 92.620 | correct, total: (670134,723532)
########################################################################
##########one 200 batch totally time cost 0.060
batchIndex 1 |trainLen 1414 | Loss: 0.178 | Acc: 93.164 | correct, total: (954,1024)
##########one 200 batch totally time cost 1.442
batchIndex 201 |trainLen 1414 | Loss: 0.176 | Acc: 92.626 | correct, total: (95797,103424)
##########one 200 batch totally time cost 2.859
batchIndex 401 |trainLen 1414 | Loss: 0.177 | Acc: 92.554 | correct, total: (190499,205824)
##########one 200 batch totally time cost 4.266
batchIndex 601 |trainLen 1414 | Loss: 0.176 | Acc: 92.600 | correct, total: (285414,308224)
##########one 200 ba

##########one 200 batch totally time cost 3.398
batchIndex 401 |trainLen 1414 | Loss: 0.174 | Acc: 92.565 | correct, total: (190520,205824)
##########one 200 batch totally time cost 4.762
batchIndex 601 |trainLen 1414 | Loss: 0.176 | Acc: 92.496 | correct, total: (285096,308224)
##########one 200 batch totally time cost 6.202
batchIndex 801 |trainLen 1414 | Loss: 0.178 | Acc: 92.449 | correct, total: (379618,410624)
##########one 200 batch totally time cost 7.626
batchIndex 1001 |trainLen 1414 | Loss: 0.177 | Acc: 92.454 | correct, total: (474312,513024)
##########one 200 batch totally time cost 9.504
batchIndex 1201 |trainLen 1414 | Loss: 0.177 | Acc: 92.506 | correct, total: (569304,615424)
##########one 200 batch totally time cost 10.979
batchIndex 1401 |trainLen 1414 | Loss: 0.178 | Acc: 92.472 | correct, total: (663789,717824)
epochIndex 92 |time: 11 | Acc: 92.474 | correct, total: (669082,723532)
########################################################################
##########o

##########one 200 batch totally time cost 10.532
batchIndex 1401 |trainLen 1414 | Loss: 0.181 | Acc: 92.383 | correct, total: (663149,717824)
epochIndex 98 |time: 10 | Acc: 92.385 | correct, total: (668437,723532)
########################################################################
##########one 200 batch totally time cost 0.065
batchIndex 1 |trainLen 1414 | Loss: 0.165 | Acc: 92.676 | correct, total: (949,1024)
##########one 200 batch totally time cost 1.885
batchIndex 201 |trainLen 1414 | Loss: 0.175 | Acc: 92.577 | correct, total: (95747,103424)
##########one 200 batch totally time cost 3.308
batchIndex 401 |trainLen 1414 | Loss: 0.179 | Acc: 92.421 | correct, total: (190225,205824)
##########one 200 batch totally time cost 4.661
batchIndex 601 |trainLen 1414 | Loss: 0.179 | Acc: 92.426 | correct, total: (284879,308224)
##########one 200 batch totally time cost 6.127
batchIndex 801 |trainLen 1414 | Loss: 0.178 | Acc: 92.454 | correct, total: (379640,410624)
##########one 200 bat

In [22]:
state = {"net": mlp.state_dict(), "optimizer": optimizer.state_dict()}
modelPathName = "./trainedModes/"+"redTLS.modeparams"
torch.save(state, modelPathName)
params = torch.load(modelPathName)
mlp.load_state_dict(params['net'])
optimizer.load_state_dict(params['optimizer'])

In [ ]:
!git add .
!git commit -m  "读写csv，MLP训练OK"
!git push

[main 5e9094a] 读写csv，MLP训练OK
 6 files changed, 2828 insertions(+), 402 deletions(-)
 rewrite mainNBDT1.ipynb (99%)
 create mode 100644 mainTestCSVMLP2-Copy1.ipynb
 create mode 100644 mainTestCSVMLP2.ipynb
 rewrite trainData/dataAllSim.zip (90%)
 create mode 100644 trainedModes/redTLS.modeparams
Enumerating objects: 27, done.
Counting objects: 100% (27/27), done.
Delta compression using up to 4 threads
Compressing objects: 100% (18/18), done.
